In [7]:
from classify_trades import call_llm

result = call_llm("test_data_features/kelly——20140713_features.json")
print(result)

千问API调用失败: Error code: 400 - {'error': {'code': 'invalid_parameter_error', 'param': None, 'message': '<400> InternalError.Algo.InvalidParameter: Range of input length should be [1, 129024]', 'type': 'invalid_request_error'}, 'id': 'chatcmpl-df78853a-b590-9586-b03c-e1b78ec96953', 'request_id': 'df78853a-b590-9586-b03c-e1b78ec96953'}
other


In [33]:
import pandas as pd

# 读取CSV文件
df_results = pd.read_csv('/home/coder/ADC2025/classification_results.csv')

# 提取 file 字段中 "——" 前的内容作为策略名
df_results['strategy'] = df_results['file'].str.split('——').str[0]

# 统计每个策略和 label 的数量关系
strategy_label_counts = df_results.groupby(['strategy', 'label']).size().unstack(fill_value=0)

print(strategy_label_counts)

label       kelly  martin  other
strategy                        
kelly           0      24     10
login           3      51      0
martingale      0      35      7
